# Generate the clusters across the landscape for holding the endemics

Each island will be 100km across, but conform to the continental boundaries.

In [3]:
import numpy as np
import pandas as pd
import os
import sys
sys.path.append("../")
from data_preparation import create_cluster, find_distance_lat_long
from pycoalescence import Map

In [4]:
# Data set up
# set this to the path to the data directory. Everything else should work from that
# data_directory = "/Volumes/Seagate 3TB/Paleo/Data"
data_directory = "/run/media/sam/Media/Paleo/Data/paleo_maps/"
# The output directory for the clustered maps
cluster_dir = os.path.join(data_directory, "clustered")
if not os.path.exists(cluster_dir):
    os.makedirs(cluster_dir)
# the list of folders containing the shape files to convert to raster
csv_directory = "../../MainSimulationR/input"
# Define the radius in kilometers
radius = 100

In [5]:
info_per_pcoord = pd.read_csv(os.path.join(csv_directory, "info_per_pcoord_occ.csv"))
info_per_pcoord['lat'] = info_per_pcoord.coord_lat
info_per_pcoord['long'] = info_per_pcoord.coord_long

In [6]:
intervals = [x for x in info_per_pcoord.interval.unique()]
tetrapod_groups = [x.lower() for x in info_per_pcoord.tetrapod_group.unique()]

In [7]:
# Calculate the origins for each of the time points
map_origins = {}
for interval in intervals:
    map_path = os.path.join(data_directory, "{}.tif".format(interval))
    paleo_mask = Map(map_path)
    _, _, _,_, x_res, y_res, ulx_mask, uly_mask = paleo_mask.get_dimensions()
    map_origins[interval] = [uly_mask, ulx_mask, y_res, x_res]

In [8]:
for interval in intervals:
    interval_map_input_path = os.path.join(data_directory, "occ_sq", "{}.tif".format(interval.lower()))
    select_pcoords = info_per_pcoord[info_per_pcoord.interval == interval]
    for tet_group in tetrapod_groups:
        interval_map_output_path = os.path.join(cluster_dir, "{}_{}.tif".format(interval.lower(), tet_group))
        if os.path.exists(interval_map_output_path):
            os.remove(interval_map_output_path)
        xs = []
        ys = []
        for i, row in select_pcoords.iterrows():
            y, x = [int(e) for e in find_distance_lat_long(map_origins[interval][0], map_origins[interval][1], 
                                                           row.lat, row.long, res=map_origins[interval][3])]
            xs.append(x)
            ys.append(y)
        coordinates = [x for x in zip(xs, ys)]
        create_cluster(coordinates=coordinates, input_map=interval_map_input_path,
                       output_map=interval_map_output_path, radius=radius)

In [15]:
a = np.random.rand(10, 10)
b = np.random.rand(10, 10)
np.max(a, b)

TypeError: only integer scalar arrays can be converted to a scalar index

In [14]:
b

array([[0.97465691, 0.45095453, 0.61975632, 0.37424827, 0.29131422,
        0.73347737, 0.46998989, 0.57166674, 0.07838719, 0.12433656],
       [0.60192862, 0.98743647, 0.2722123 , 0.85380427, 0.40711343,
        0.6564667 , 0.54542969, 0.97161552, 0.4269897 , 0.5510039 ],
       [0.45156019, 0.45708422, 0.53549193, 0.40402225, 0.44607077,
        0.30912279, 0.38814365, 0.17718329, 0.57219655, 0.03759111],
       [0.00648354, 0.82661052, 0.11999906, 0.94806962, 0.16444646,
        0.60645498, 0.83797541, 0.90395874, 0.3378491 , 0.61305285],
       [0.71572533, 0.00224703, 0.40694988, 0.79849569, 0.35636942,
        0.50256834, 0.03675803, 0.68326731, 0.36919683, 0.72748628],
       [0.75542473, 0.43766262, 0.05772334, 0.15206638, 0.18842162,
        0.33667431, 0.06306786, 0.85986403, 0.72574189, 0.15299518],
       [0.20072242, 0.61779931, 0.52268839, 0.89785047, 0.89215624,
        0.06279325, 0.90261163, 0.64305676, 0.36103358, 0.85189005],
       [0.42638958, 0.93308081, 0.0074322